In [1]:
import os

os.chdir("..")

In [2]:
from neuralforecast.models import NBEATS
from neuralforecast import NeuralForecast
from utils import load_dataset, train_test_split
from models.NBeatsMoe import NBeatsMoe, NBEATSMoEBlock
import torch.nn as nn
import pandas as pd

# datasets list
datasets = [
    {
        "name": "m3",
        "directory": "C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3\\",
        "group": "Monthly",
        "freq": "M",
    },
    # {
    #     "name": "m4",
    #     "directory": "C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m4\\",
    #     "group": "Monthly",
    #     "freq": "M",
    # } 
]

/home/ricardo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-19 23:42:28,787	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-19 23:42:28,874	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# Y_ALL_m4 = load_dataset("m4", datasets[1])
# Y_ALL_m3 = load_dataset("m3", datasets[0])
n_lags = 36
horizon = 18

Let's define the models and the gates to evaluate ...

In [4]:
from neuralforecast.losses.numpy import smape

def calculate_smape(Y_test, predictions, model_name):
    y_true = Y_test['y'].values
    y_hat = predictions[model_name].values

    n_series = Y_test['unique_id'].nunique()
    y_true = y_true.reshape(n_series, -1)
    y_hat = y_hat.reshape(n_series, -1)

    smape_value = smape(y_true, y_hat)
    return smape_value

In [ ]:
gate_type_list = ["conv1d-gap", "conv1d-flatten", "no_gate", "linear", "mlp", "conv1d", ]


for gate_type in gate_type_list:
    for dataset in datasets:
        
        # Load dataset
        Y_ALL = load_dataset(dataset["name"], dataset)
        # Train test split
        Y_train, Y_test = train_test_split(Y_ALL, horizon)
        # Define model
        if gate_type == "no_gate":
            model = NBEATS(
                h=horizon,
                input_size=n_lags,
                mlp_units=3*[[128,128]],
                max_steps=5000,
                early_stop_patience_steps=10,
            )
        else:
            model = NBeatsMoe(
                h=horizon,
                input_size=n_lags,
                nr_experts=4,
                top_k=2,
                mlp_units=3*[[128,128]],
                gate_type=gate_type,
                max_steps=5000,
                early_stop_patience_steps=10,
            )
        # Define NeuralForecast
        nf = NeuralForecast(models=[model], freq=dataset["freq"])
        # Fit
        nf.fit(df=Y_train, static_df=None, val_size=horizon)
        # Predict
        predictions = nf.predict(futr_df=Y_test)
        
        smape_value = calculate_smape(Y_test, predictions, model_name=model.__class__.__name__)
        print(f"Dataset: {dataset['name']}\nGate Type: {gate_type}\nSMAPE: {smape_value}")
    

Loading m3_monthly dataset...


Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2025-04-19 23:42:39.752320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745102559.768014   40099 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745102559.772728   40099 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 23:42:39.791585: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 39:  64%|██████▍   | 29/45 [00:25<00:14,  1.14it/s, v_num=66, train_loss_step=441.0, train_loss_epoch=488.0, valid_loss=654.0]